In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
from ideas import intersection_over_union
from ideas import al
from ideas import aw

box_columns = ['Xmin','Ymin','Xmax','Ymax']
box_columns_gd = ['Xmin_gd','Ymin_gd','Xmax_gd','Ymax_gd']

def weighted_averaging(iid, df, np_data, avg):
    items = df[df['itemId'] == iid]
    vi = np_data[items.index.values]
    if vi.shape[0] == 0:
        return
    l = al(vi)
    w = aw(vi)
    sabx = (l * w) ** 2
    sb = sum(1 / sabx)
    for y in range(0, 4):        
        avg.loc[iid][y] = sum(vi[:,y] / sabx) / sb
    avg.loc[iid][4] = iid
    return avg.loc[iid]

codes = [
    Path.MOVETO,
    Path.LINETO,
    Path.LINETO,
    Path.LINETO,
    Path.CLOSEPOLY
]

In [15]:
te_d = pd.read_csv('test_data.csv')
t_a = pd.read_csv('train_answers.csv')
ids = list(set(te_d['itemId']))
te_d['area'] = (te_d['Xmax'] - te_d['Xmin'])*(te_d['Ymax'] - te_d['Ymin'])
te_d_max = te_d[['area', 'itemId']].groupby('itemId').max()
te_d = te_d.merge(te_d_max, on=['itemId'])
te_d['area_ratio'] = te_d['area_x']/te_d['area_y']
te_d_r = te_d[te_d['area_ratio'] > .75][box_columns + ['itemId']]
te_d_r = te_d_r.reset_index()
np_te_d = te_d_r[box_columns].values

In [16]:
te_d_wa = pd.DataFrame(np.zeros((len(ids), 5), dtype=int), 
                      index=ids, columns=[box_columns_gd + ['itemId']])
for iid in ids:
    weighted_averaging(iid, te_d_r, np_te_d, te_d_wa)
    
te_d_wa.to_csv('test_data_wavg.csv', index=False)

In [17]:
data = te_d_wa.merge(t_a, on=['itemId'])
data["iou"] = data[box_columns_gd + ['Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
print(data["iou"].mean()) 

ValueError: The column label 'itemId' is not unique.
For a multi-index, the label must be a tuple with elements corresponding to each level.